In [ ]:
import sys, os
import cv2
import numpy as np
import pose_utils
import json
import matplotlib.pyplot as plt
from PIL import Image

## Update the path to all your data

In [ ]:
# path to 3D model
object_path = r"example\path\example_object.ply"
# path to camera intrinsics
camera_path = r"example\path\camera_params.json"
# Path to images/video
data_path = r"example\path\video.mp4"
# Where to store all the data
output_path = r"example\path\output"

In [ ]:
# load 3D model
mesh                = pose_utils.MeshPly(object_path)
vertices_og         = np.c_[np.array(mesh.vertices)/1000, np.ones((len(mesh.vertices), 1))].transpose() # vertices in object coordinate in meters
corners3D           = pose_utils.get_3D_corners(vertices_og)
vertices            = np.hstack((np.array([0,0,0,1]).reshape(4,1), corners3D)) # add center coordinate

In [ ]:
# load camera params
with open(camera_path, 'r') as f:
    camera_data = json.load(f)

# camera_model = None
dtx = np.array(camera_data["distortion"])
mtx = np.array(camera_data["intrinsic"])

### Set the offset from the charuco frame to the center of your object.
* Carefully measure the offset from the top left corner of your charuco board to the center of your object. The more accurate, the better

In [ ]:
# Predefined offset from charuco frame to object frame
rotation_offset = [90.0, 0.0, 0.0] # degrees
translation_offset = [0.22, 0.1485, -0.04075] # translation in meters #  [0.221, 0.14075, -0.0485 ]

offset_mat = pose_utils.construct_transform(translation_offset, rotation_offset)

### Steps to acquire object pose and save labels

1. Run through all images 
2. Detect charuco pose
3. Determine offset from charuco pose to object pose (in charuco frame coordinates)
4. Project object onto image
5. Draw projected object and calculate mask
6. Store all information

In [ ]:
# 1. Run through video
cap = cv2.VideoCapture(data_path)
frame_count = 0

while(cap.isOpened()):

    ret, frame = cap.read()
    if not ret:
        break

    print(f"Frame {frame_count}")
    # 2. detect charuco pose
    frame_remapped_gray = frame[:, :, 0]
    im_height, im_width = frame_remapped_gray.shape
    # print(frame.shape)
    im_with_charuco_board, pose = pose_utils.detect_Charuco_pose_board(frame, mtx, dtx) # update based on your own charuco board layout
    if pose != None:
        
        # 3. determine offset from charuco pose to object pose (in charuco frame coordinates)
        rvec = pose['rvec']
        tvec = pose['tvec']
        rotation = cv2.Rodrigues(rvec)[0]
        transform_mat = np.vstack((np.hstack((rotation, tvec)), np.array([0, 0, 0, 1])))
        transform_mat = np.matmul(transform_mat, offset_mat)
        # 4. project object onto image
        projected_corners = pose_utils.compute_projection(corners3D, transform_mat[:3, :], mtx)
        projected_vertices = pose_utils.compute_projection(vertices, transform_mat[:3, :], mtx)
        # 5. Draw projected object
        im_with_charuco_board = pose_utils.draw_BBox(im_with_charuco_board, projected_corners.T, projected_vertices.T)
        # Create mask
        mask_arr = pose_utils.create_simple_mask(projected_vertices.T, im_width, im_height)
        # Create label
        label = pose_utils.create_label(0, projected_vertices, mtx[0,0],  mtx[1,1] , im_width, im_height, mtx[0,2], mtx[1,2], im_width, im_height, transform_mat)
        imageName = f"frame_{frame_count}.png"
        # 6. store all information
        pose_utils.save_data(frame, mask_arr, label, imageName, output_path, im_with_charuco_board )

    frame_count += 1

# Sanity Check

After creating your dataset, it is always good to run a sanity check

In [ ]:
## Here a couple fo functions to help you visualize the data

# Load first image from data
img_path = os.path.join(output_path, "images")
label_path = os.path.join(output_path, "labels")
mask_path = os.path.join(output_path, "mask")

for file in os.listdir(img_path):
    if file.endswith(".png"):
        assert os.path.isfile(os.path.join(label_path, file[:-4] + ".txt")), f"Label file {file[:-4] + '.txt'} does not exist"
        assert os.path.isfile(os.path.join(mask_path, file)), f"Mask file {file} does not exist"

        # Load image, label and mask
        img = cv2.imread(os.path.join(img_path, file))
        label = np.loadtxt(os.path.join(label_path, file[:-4] + ".txt"))
        mask = cv2.imread(os.path.join(mask_path, file))
        break

keypoints = label[1:19]
keypoints_x = keypoints[::2]*img.shape[1]
keypoints_y = keypoints[1::2]*img.shape[0]


# create figure and axes
fig,ax = plt.subplots(1)
# display the image
ax.imshow(img)
# scatter points
ax.scatter(keypoints_x, keypoints_y, s=10, c='red', marker='o')
plt.show()